In [9]:
import pandas as pd
import numpy as np
import xgboost as xgb
import shap
pd.set_option('display.max_columns', None)
df_income_race = pd.read_csv(r"input_data\df_withincome_race.csv")
df_visit = pd.read_csv(r"input_data\visiting_pattern.csv")

df_income_race.head()
df_visit.head()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2912212\1848429539.py:7: DtypeWarning: Columns (3,4,6,19,29,100) have mixed types. Specify dtype option on import or set low_memory=False.
  df_visit = pd.read_csv(r"input_data\visiting_pattern.csv")


,Unnamed: 0.1,Unnamed: 0,PLACEKEY,PARENT_PLACEKEY_x,SAFEGRAPH_BRAND_IDS_x,LOCATION_NAME_x,BRANDS_x,STORE_ID_x,TOP_CATEGORY_x,SUB_CATEGORY_x,NAICS_CODE_x,LATITUDE_x,LONGITUDE_x,STREET_ADDRESS_x,CITY_x,REGION_x,POSTAL_CODE_x,OPEN_HOURS_x,CATEGORY_TAGS_x,OPENED_ON_x,CLOSED_ON_x,TRACKING_CLOSED_SINCE_x,WEBSITES,GEOMETRY_TYPE_x,POLYGON_WKT_x,POLYGON_CLASS_x,ENCLOSED_x,PHONE_NUMBER_x,IS_SYNTHETIC_x,INCLUDES_PARKING_LOT_x,ISO_COUNTRY_CODE_x,WKT_AREA_SQ_METERS_x,DATE_RANGE_START,DATE_RANGE_END,RAW_VISIT_COUNTS,RAW_VISITOR_COUNTS,VISITS_BY_DAY,VISITS_BY_EACH_HOUR,POI_CBG,VISITOR_HOME_CBGS,VISITOR_HOME_AGGREGATION,VISITOR_DAYTIME_CBGS,VISITOR_COUNTRY_OF_ORIGIN,DISTANCE_FROM_HOME,MEDIAN_DWELL,BUCKETED_DWELL_TIMES,RELATED_SAME_DAY_BRAND,RELATED_SAME_WEEK_BRAND,DEVICE_TYPE,NORMALIZED_VISITS_BY_TOTAL_VISITORS,geometry_x,in1km_x,VISIT_BY_DAY_list,date_xh,date_xhd,dow_x,tract,population,income,poverty_ratio,lessedu_ratio,female_ratio,white_ratio,black_ratio,asian_ratio,walk_work,transit_work,drive_work,SPEND_DATE_RANGE_START_x,SPEND_DATE_RANGE_END_x,RAW_TOTAL_SPEND_x,RAW_NUM_TRANSACTIONS_x,RAW_NUM_CUSTOMERS_x,MEDIAN_SPEND_PER_TRANSACTION_x,MEDIAN_SPEND_PER_CUSTOMER_x,SPEND_PER_TRANSACTION_PERCENTILES_x,SPEND_BY_DAY_x,SPEND_PER_TRANSACTION_BY_DAY_x,SPEND_BY_DAY_OF_WEEK_x,SPEND_PCT_CHANGE_VS_PREV_MONTH_x,SPEND_PCT_CHANGE_VS_PREV_YEAR_x,ONLINE_TRANSACTIONS_x,ONLINE_SPEND_x,TRANSACTION_INTERMEDIARY_x,SPEND_BY_TRANSACTION_INTERMEDIARY_x,BUCKETED_CUSTOMER_FREQUENCY_x,MEAN_SPEND_PER_CUSTOMER_BY_FREQUENCY_x,BUCKETED_CUSTOMER_INCOMES_x,MEAN_SPEND_PER_CUSTOMER_BY_INCOME_x,CUSTOMER_HOME_CITY_x,RELATED_CROSS_SHOPPING_PHYSICAL_BRANDS_PCT_x,RELATED_CROSS_SHOPPING_ONLINE_MERCHANTS_PCT_x,RELATED_CROSS_SHOPPING_SAME_CATEGORY_BRANDS_PCT_x,RELATED_CROSS_SHOPPING_LOCAL_BRANDS_PCT_x,RELATED_WIRELESS_CARRIER_PCT_x,RELATED_STREAMING_CABLE_PCT_x,RELATED_DELIVERY_SERVICE_PCT_x,RELATED_RIDESHARE_SERVICE_PCT_x,RELATED_BUYNOWPAYLATER_SERVICE_PCT_x,RELATED_PAYMENT_PLATFORM_PCT_x,BRANDS_x.1,DOMAINS,WEBSITE,date,VISIT_BY_DAY_normalized
0,0,0,223-222@62j-sz8-gtv,NaN,NaN,Wellington's Pizzeria,NaN,NaN,Restaurants and Other Eating Places,Full-Service Restaurants,722511,42.418166,-71.108893,22 Riverside Ave,Medford,MA,2155.0,"{ ""Mon"": [[""11:00"", ""22:00""]], ""Tue"": [[""11:00...","Italian Food,Pizza",NaN,NaN,2019-07-01 00:00:00.000,wellingtonspizzeria.com,POLYGON,POLYGON ((-71.1089471639325 42.418119505802615...,OWNED_POLYGON,False,1.781391e+10,False,False,US,65.0,2022-05-30 00:00:00.000,2022-06-06 00:00:00.000,10.0,9.0,"[3,2,0,1,1,2,1]","[0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0...",250173391004,"{""250173563006"":4}","{""25017356300"":4,""25017339100"":4}","{""250173391001"":4}","{""US"":9}",3260.0,5.0,"{ ""<5"":9, ""5-20"":1, ""21-60"":0, ""61-240"":0, "">2...","{""Lifestance Health"":2,""Mass General Brigham"":...","{""CVS"":6,""Western Union"":6,""Starbucks"":5,""Targ...","{""ios"":8}",0.000044,POINT (232189.47499430945 907590.3078673852),False,0,2022-06-01,2022-06-01,3,25017339100,6504,40.07,0.074266,0.087952,0.539053,0.886839,0.030443,0.03198,0.028395,0.125617,0.663272,2022-06-01 00:00:00.000,2022-07-01 00:00:00.000,49.47,2,2,24.74,24.74,"{""25"":22.45,""75"":27.02}",0.00,0.00,"{""Monday"":0.00,""Tuesday"":0.00,""Wednesday"":27.0...",NaN,NaN,0,0.0,"{""No intermediary"":2}","{""No intermediary"":49.47}","{""1"":2,""2"":0,""3"":0,""4"":0,""5-10"":0,"">10"":0}","{""1"":24.74,""2"":null,""3"":null,""4"":null,""5-10"":n...","{""60-75k"":2,""100-150k"":3}","{""60-75k"":27.02,""100-150k"":22.45}","{""Erda, UT"":2,""Medford, MA"":2}","{""Star Market"":50,""L.L.Bean"":50,""Trader Joe's""...","{""Uber"":50,""Amazon"":50,""Delta Air Lines"":50,""N...","{""The Boiling Crab"":50,""B.GOOD"":50,""Starbucks""...","{""Target"":50,""Wegmans Food Markets"":50,""Whole ...","{""Verizon Wireless"":50}","{""Netflix"":50}",{},"{""Uber"":50}",{},{},NaN,"""[\""wellingtonspizzeria.com\""]""",http://www.wellingtonspizzeria.com/,2022-06-01,-0.292587
1,1,1,223-222@62j-sz8-gtv,NaN,NaN,Wellin

In [10]:
df_visit.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'PLACEKEY', 'PARENT_PLACEKEY_x',
       'SAFEGRAPH_BRAND_IDS_x', 'LOCATION_NAME_x', 'BRANDS_x', 'STORE_ID_x',
       'TOP_CATEGORY_x', 'SUB_CATEGORY_x',
       ...
       'RELATED_STREAMING_CABLE_PCT_x', 'RELATED_DELIVERY_SERVICE_PCT_x',
       'RELATED_RIDESHARE_SERVICE_PCT_x',
       'RELATED_BUYNOWPAYLATER_SERVICE_PCT_x',
       'RELATED_PAYMENT_PLATFORM_PCT_x', 'BRANDS_x.1', 'DOMAINS', 'WEBSITE',
       'date', 'VISIT_BY_DAY_normalized'],
      dtype='object', length=105)

In [11]:
df_visit_cleaned = df_visit[['PLACEKEY','VISIT_BY_DAY_list','date_xh']]

In [12]:
before_mean = df_visit_cleaned[df_visit_cleaned['date_xh'] < '2022-12-12'].groupby('PLACEKEY')['VISIT_BY_DAY_list'].mean()

after_mean = df_visit_cleaned[df_visit_cleaned['date_xh'] >= '2022-12-12'].groupby('PLACEKEY')['VISIT_BY_DAY_list'].mean()

visit_difference = after_mean - before_mean

new_df = pd.DataFrame({'PLACEKEY': visit_difference.index, 'visit_difference': visit_difference.values})

,PLACEKEY,visit_difference
0,222-222@62j-sgg-gkz,3.107978
1,222-222@62j-sgg-kfz,4.778386
2,222-222@62j-sj2-bzf,-2.745455
3,222-222@62j-sj2-snq,4.857547
4,222-222@62j-sj2-t35,3.432773
...,...,...
899,zzy-228@62j-sk2-vs5,-11.756582
900,zzy-229@62j-sk2-vs5,3.407657
901,zzy-22b@62j-sk2-vmk,5.191393
902,zzy-22d@62j-sj4-vmk,-16.609212


In [20]:
before_mean = df_income_race[df_income_race['DATE_RANGE_END'] < '2022-12-12'].groupby('PLACEKEY')[['race_seg', 'income_seg']].mean()

after_mean = df_income_race[df_income_race['DATE_RANGE_END'] >= '2022-12-12'].groupby('PLACEKEY')[['race_seg', 'income_seg']].mean()

differences = after_mean - before_mean

differences.rename(columns={'race_seg': 'race_difference', 'income_seg': 'income_difference'}, inplace=True)
new_income_race = pd.merge(differences, df_income_race[['PLACEKEY', 'population', 'income', 'poverty_ratio', 'female_ratio', 'white_ratio', 'black_ratio', 'asian_ratio', 'walk_work', 'transit_work', 'drive_work']], on='PLACEKEY')

new_income_race.drop_duplicates(subset='PLACEKEY', inplace=True)

,PLACEKEY,race_difference,income_difference,population,income,poverty_ratio,female_ratio,white_ratio,black_ratio,asian_ratio,walk_work,transit_work,drive_work
0,222-222@62j-sgg-gkz,-0.281481,-0.309259,5131,37.132,0.167552,0.478074,0.644319,0.088482,0.199961,0.331226,0.261367,0.254743
16,222-222@62j-sgg-kfz,-0.238085,-0.291075,2977,41.742,0.127309,0.484380,0.516292,0.259321,0.112193,0.206400,0.396800,0.166933
73,222-222@62j-sj2-bzf,-0.425872,0.048468,2623,34.005,0.178101,0.494853,0.630194,0.199390,0.126573,0.324392,0.355583,0.220212
110,222-222@62j-sj2-snq,-0.276823,-0.303300,3609,41.786,0.171341,0.497368,0.641175,0.124411,0.164311,0.116816,0.502120,0.165332
150,222-222@62j-sj2-t35,-0.225185,-0.253333,3609,41.786,0.171341,0.497368,0.641175,0.124411,0.164311,0.116816,0.502120,0.165332
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33304,zzy-228@62j-sk2-vs5,-0.006252,-0.142928,2777,53.419,0.088225,0.514224,0.789341,0.060857,0.141520,0.108125,0.474728,0.236724
33367,zzy-229@62j-sk2-vs5,0.000000,0.000000,2777,53.419,0.088225,0.514224,0.789341,0.060857,0.141520,0.108125,0.474728,0.236724
33374,zzy-22b@62j-sk2-vmk,-0.122954,-0.185274,2777,53.419,0.088225,0.514224,0.789341,0.060857,0.141520,0.108125,0.474728,0.236724
33434,zzy-22d@62j-sj4-vmk,-0.136048,0.018751,2075,30.551,0.211566,0.547470,0.383614,0.360482,0.122410,0.215985,0.406280,0.254044


In [24]:
df_income_race_with_differences = pd.merge(new_df, new_income_race, on='PLACEKEY')
df_income_race_with_differences['log_population'] = np.log(df_income_race_with_differences['population'])
df_income_race_with_differences['log_income'] = np.log(df_income_race_with_differences['income'] * 1000)

In [23]:
def train_xgb_model(X, y):
    params = {
        'objective': 'reg:squarederror',
        'learning_rate': 0.1,
        'max_depth': 6,
        'min_child_weight': 1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'n_estimators': 100
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X, y)
    return model

def calculate_marginal_effects(model, X, feature_names):
    results = []
    
    for feature in feature_names:
        X_base = X.copy()
        base_pred = model.predict(X_base)

        X_unit = X_base.copy()
        X_unit[feature] = X_unit[feature] + 1
        unit_pred = model.predict(X_unit)
        unit_effect = (unit_pred - base_pred).mean()

        q25 = X[feature].quantile(0.25)
        q75 = X[feature].quantile(0.75)
        
        X_q25 = X_base.copy()
        X_q75 = X_base.copy()
        X_q25[feature] = q25
        X_q75[feature] = q75
        
        q25_pred = model.predict(X_q25)
        q75_pred = model.predict(X_q75)
        percentile_effect = (q75_pred - q25_pred).mean()
        
        results.append({
            'feature': feature,
            'unit_effect': unit_effect,
            'percentile_effect': percentile_effect
        })
    
    return pd.DataFrame(results)

def create_marginal_effects_table(df, features, outcome_vars):
    all_results = []
    
    for outcome in outcome_vars:
        print(f"\nProcessing outcome variable: {outcome}")

        X = df[features].copy()
        y = df[outcome].copy()
        valid_idx = ~(X.isna().any(axis=1) | y.isna())
        X = X[valid_idx]
        y = y[valid_idx]

        model = train_xgb_model(X, y)

        effects = calculate_marginal_effects(model, X, features)
        effects['outcome'] = outcome
        all_results.append(effects)

    final_results = pd.concat(all_results, axis=0)

    table = pd.pivot_table(
        final_results,
        values=['unit_effect', 'percentile_effect'],
        index=['feature'],
        columns=['outcome']
    )

    new_columns = []
    for col in table.columns:
        effect_type = 'One-unit increase' if col[0] == 'unit_effect' else 'Switch from low to high'
        outcome_name = col[1]
        new_columns.append(f"{outcome_name} - {effect_type}")
    table.columns = new_columns
    
    return table

def main():
    df = df_income_race_with_differences

    features = [
        'log_population', 'log_income', 'poverty_ratio', 'female_ratio',
        'white_ratio', 'black_ratio', 'asian_ratio', 'walk_work',
        'transit_work', 'drive_work'
    ]
    
    outcome_vars = [
        'visit_difference',
        'income_difference',
        'race_difference'
    ]

    print("Calculating marginal effects...")
    results_table = create_marginal_effects_table(df, features, outcome_vars)

    pd.set_option('display.float_format', lambda x: '%.3f' % x)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)

    print("\nMarginal Effects Table:")
    print(results_table)
    
    return results_table

if __name__ == "__main__":
    results = main()

Calculating marginal effects...

Processing outcome variable: visit_difference

Processing outcome variable: income_difference

Processing outcome variable: race_difference

Results saved to 'marginal_effects_table.csv'

Marginal Effects Table:
               income_difference - Switch from low to high  \
feature                                                      
asian_ratio                                         -0.002   
black_ratio                                         -0.012   
drive_work                                           0.015   
female_ratio                                         0.024   
income                                               0.001   
population                                          -0.009   
poverty_ratio                                       -0.008   
transit_work                                        -0.006   
walk_work                                           -0.001   
white_ratio                                          0.034   

          